In [0]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [3]:
# インスタンス起動からの時間を表示(0.5days: 12hでセッションの有無にかかわらずシャットダウンされる)
# タブがアイドル状態で90mが経過してもシャットダウンされる
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.0126948days (1096.83sec)


In [4]:
# 残りのディスク容量を確認
!df -h

Filesystem              Size  Used Avail Use% Mounted on
overlay                 359G   26G  315G   8% /
tmpfs                   6.4G     0  6.4G   0% /dev
tmpfs                   6.4G     0  6.4G   0% /sys/fs/cgroup
/dev/sda1               365G   38G  328G  11% /opt/bin
tmpfs                   6.4G   12K  6.4G   1% /var/colab
shm                     6.0G     0  6.0G   0% /dev/shm
tmpfs                   6.4G     0  6.4G   0% /sys/firmware
google-drive-ocamlfuse  100G   25G   76G  25% /content/drive


In [5]:
# メインメモリの利用状況を確認(70~80%を超えるとダイアログで警告される)
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        438M         11G        908K        750M         12G
Swap:            0B          0B          0B


In [6]:
# GPUメモリの利用状況確認
!nvidia-smi

Wed Jun 12 07:43:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
# GoogleDriveマウント用パッケージ
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
# Googledriveへのアクセス許可(トークン)
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [5]:
# アクセス許可(2)
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [6]:
# Google drive の特定フォルダのみマウント
# root_folder= はマウントしたいフォルダのURLの末尾
!cp -f ~/.gdfuse/default/config config
!sed -i -e "s/^root_folder=$/root_folder=" config

!mkdir -p drive
!google-drive-ocamlfuse -config ./config -cc drive
!ls drive

sed: -e expression #1, char 29: unterminated `s' command
Clearing cache...done
 backups		     mienai_tikara.ZIP	      お仕事
 code			     mnt		      修正前Melty-Fantasia.mp3
'Colab Notebooks'	     mysql		      制作物
 Coursera		     Profile		      勉強会
 cycling		     raspi_craft	      各種スライド
 darwin			     RCC		      絵の参考
 DeepGenerativeModel勉強会   Ritsumei_TapTap	     'スノハレ(背景付き).png'
 ds_specialcontents	     SECCON_beginers_Nagoya   読み物
 DzWlScMUwAAuyq1.jpg	     yaruodsp.pdf	      論文
 exp3			     zunda.JPG


In [2]:
# ファイルのアップロード
from google.colab import files
uploaded = files.upload()

Saving util_for_colab.py to util_for_colab.py


In [59]:
!ls tf-layers-model

checkpoint			  model.ckpt-2.data-00000-of-00001
model.ckpt-1.data-00000-of-00001  model.ckpt-2.index
model.ckpt-1.index		  model.ckpt-2.meta
model.ckpt-1.meta


In [0]:
# ファイルのダウンロード
files.download('./tf-layers-model/model.ckpt-1.index')

In [3]:
import tensorflow as tf
import numpy as np
import const_for_colab as C
import util_for_colab as util
import os

# GPUがアサインされているか確認
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# tf.nnのfilter = [kernel_height, kernel_width, input_channel, output_channel]
# 畳み込み層(エンコーダ部分)
def conv2d(x, output_channel, kernel=5, stride=2, pad='same', batch_norm=True, is_train=True, leaky_relu=True):
    net = tf.layers.conv2d(x,
                           filters=output_channel,
                           kernel_size=[kernel, kernel],
                           strides=[stride, stride],
                           padding=pad)
    if leaky_relu:
        net = tf.nn.leaky_relu(net, 0.2)
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_train)
    return net

In [0]:
# tf.nnのfilter = [kernel_height, kernel_width, output_channel, input_channel]
# output_shape = [バッチ数, 得たいheight, 得たいwidth, 得たいchannel]
# 逆畳み込み層(デコーダ部分)
def de_conv2d(x, output_channel, kernel=5, stride=2, pad='same', batch_norm=True, is_train=True, relu=True):
    net = tf.layers.conv2d_transpose(x,
                                     filters=output_channel,
                                     kernel_size=[kernel, kernel],
                                     strides=[stride, stride],
                                     padding=pad)
    if relu:
        net = tf.nn.relu(net)
    if batch_norm:
        net = tf.layers.batch_normalization(net, training=is_train)
    return net

In [0]:
# ミニバッチ毎に処理する関数(ジェネレータを返す)
def batch_generator(X, y, batch_size=C.BATCH_SIZE, shuffle=False, random_seed=None):
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
        
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size]) # 段階的に返す

In [0]:
class UNet(object):
    def __init__(self, batchsize=C.BATCH_SIZE, epochs=30, learning_rate=1e-4,
                 dropout_rate=0.5, shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
        
        g = tf.Graph()
        with g.as_default():
            tf.set_random_seed(random_seed)
            self.build()
            self.init_op = tf.global_variables_initializer()  # 変数を初期化
            self.saver = tf.train.Saver()
        
        # セッションを作成し，計算グラフgを渡す
        self.sess = tf.Session(graph=g)
    
    def build(self):
        is_train = tf.placeholder(tf.bool, shape=(), name='is_train')
        tf_x = tf.placeholder(tf.float32, shape=[None, 512, 128, 1], name='tf_x')
        tf_y = tf.placeholder(tf.float32, shape=[None, 512, 128, 1], name='tf_y')
        conv1 = conv2d(tf_x, output_channel=16)
        conv2 = conv2d(conv1, output_channel=32)
        conv3 = conv2d(conv2, output_channel=64)
        conv4 = conv2d(conv3, output_channel=128)
        conv5 = conv2d(conv4, output_channel=256)
        conv6 = conv2d(conv5, output_channel=512)
        de_conv1 = de_conv2d(conv6, output_channel=256)
        concat1 = tf.concat([de_conv1, conv5], axis=-1)
        dropout1 = tf.layers.dropout(concat1, rate=self.dropout_rate, training=is_train)
        de_conv2 = de_conv2d(dropout1, output_channel=128)
        concat2 = tf.concat([de_conv2, conv4], axis=-1)
        dropout2 = tf.layers.dropout(concat2, rate=self.dropout_rate, training=is_train)
        de_conv3 = de_conv2d(dropout2, output_channel=64)
        concat3 = tf.concat([de_conv3, conv3], axis=-1)
        dropout3 = tf.layers.dropout(concat3, rate=self.dropout_rate, training=is_train)
        de_conv4 = de_conv2d(dropout3, output_channel=32)
        concat4 = tf.concat([de_conv4, conv2], axis=-1)
        de_conv5 = de_conv2d(concat4, output_channel=16)
        concat5 = tf.concat([de_conv5, conv1], axis=-1)
        de_conv6 = de_conv2d(concat5, output_channel=1, relu=False)
        activation_final = tf.math.sigmoid(de_conv6)
        
        # 損失関数
        pred = tf_x * activation_final  # マスクをかけた元音源
        loss = tf.reduce_mean(
                tf.map_fn(tf.abs, (pred - tf_y)),
                name='mean_absolute_error')
        # 最適化
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(loss, name='train_op')
        
    def save(self, epoch, path='./tf-layers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)

        print('Saving model in %s' % path)
        self.saver.save(self.sess,
                        os.path.join(path, 'model.ckpt'),
                        global_step=epoch)

    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess,
                           os.path.join(path, 'model.ckpt-%d' % epoch))

    def train(self, X_list, y_list, initialize=True, subepoch=None):
        # 変数を初期化
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []
#         X_data = np.array(training_set[0])
#         y_data = np.array(training_Set[1])
        item_count = len(X_list)
        item_length = [x.shape[1] for x in X_list]
        subepoch = sum(item_length) // C.PATCH_LENGTH // C.BATCH_SIZE * 4

        for epoch in range(1, self.epochs + 1):
#             batch_gen = batch_generator(X_data, y_data, shuffle=self.shuffle)
            avg_loss = 0.0

            for i in range(subepoch):
                print('subepoch: ' + str(i))
                X = np.zeros((C.BATCH_SIZE, 512, C.PATCH_LENGTH, 1),
                              dtype="float32")
                Y = np.zeros((C.BATCH_SIZE, 512, C.PATCH_LENGTH, 1),
                              dtype="float32")
                idx_item = np.random.randint(0, item_count, C.BATCH_SIZE)
                for i in range(C.BATCH_SIZE):
                    randidx = np.random.randint(
                        item_length[idx_item[i]]-C.PATCH_LENGTH-1)
                    X[i, :, :, 0] = \
                        X_list[idx_item[i]][1:, randidx:randidx+C.PATCH_LENGTH]
                    Y[i, :, :, 0] = \
                        y_list[idx_item[i]][1:, randidx:randidx+C.PATCH_LENGTH]
                    feed = {'tf_x:0': X,
                            'tf_y:0': Y,
                            'is_train:0': True} # ドロップアウト
#                 for k, (batch_x, batch_y) in enumerate(batch_gen):
#                     feed = {'tf_x:0': batch_x,
#                             'tf_y:0': batch_y,
#                             'is_train:0': True} # ドロップアウト
                    loss, _ = self.sess.run(['mean_absolute_error:0', 'train_op'],
                                            feed_dict=feed)
                    avg_loss += loss

            print('Epoch %2d: Training Avg. Loss: %7.3f' % (epoch, avg_loss))
            model.save(epoch=epoch)

In [0]:
X_list, y_list = util.load_dataset('drive/mnt/spectrograms', target="vocal")
item_count = len(X_list)
item_length = [x.shape[1] for x in X_list]

In [9]:
print(X_list)
print(y_list)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[1.2921836e-03, 4.1606007e-04, 3.5980094e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [7.5067644e-04, 7.3673652e-04, 5.5286422e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.7291037e-04, 7.5295247e-04, 4.7082786e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [3.0378287e-06, 3.8120393e-06, 9.4644311e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.3535156e-08, 4.6592600e-06, 3.5130465e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.6555115e-07, 5.0523086e-06, 1.2459036e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32), array([[0.0000000e+00, 2.2095183e-04, 6.7

In [10]:
print(item_count)
print(len(item_length))
print(item_length)

80
80
[20504, 14272, 14089, 17272, 18182, 23104, 18960, 19891, 19057, 21553, 25384, 20056, 30758, 14535, 16538, 23565, 27329, 12203, 17698, 20857, 22074, 24634, 16508, 22790, 16436, 25222, 23256, 21251, 29606, 22373, 19330, 18282, 27380, 21628, 15280, 34080, 21806, 22126, 26714, 22451, 21057, 17429, 21796, 15147, 36302, 16155, 18097, 17907, 14475, 20189, 18098, 17714, 16952, 15229, 15469, 18897, 29031, 21957, 34344, 23758, 25994, 21890, 16407, 16107, 17090, 27162, 37070, 17390, 13721, 19040, 22266, 21823, 22009, 23054, 19983, 24257, 19590, 17129, 23443, 21374]


In [11]:
subepoch = sum(item_length) // C.PATCH_LENGTH // C.BATCH_SIZE * 4
print(subepoch)

820


In [0]:
model = UNet(random_seed=42)

In [55]:
model.train(X_list, y_list, subepoch=subepoch)

subepoch: 0
subepoch: 1
subepoch: 2
subepoch: 3
subepoch: 4
subepoch: 5
subepoch: 6
subepoch: 7
subepoch: 8
subepoch: 9
subepoch: 10
subepoch: 11
subepoch: 12
subepoch: 13
subepoch: 14
subepoch: 15
subepoch: 16
subepoch: 17
subepoch: 18
subepoch: 19
subepoch: 20
subepoch: 21
subepoch: 22
subepoch: 23
subepoch: 24
subepoch: 25
subepoch: 26
subepoch: 27
subepoch: 28
subepoch: 29
subepoch: 30
subepoch: 31
subepoch: 32
subepoch: 33
subepoch: 34
subepoch: 35
subepoch: 36
subepoch: 37
subepoch: 38
subepoch: 39
subepoch: 40
subepoch: 41
subepoch: 42
subepoch: 43
subepoch: 44
subepoch: 45
subepoch: 46
subepoch: 47
subepoch: 48
subepoch: 49
subepoch: 50
subepoch: 51
subepoch: 52
subepoch: 53
subepoch: 54
subepoch: 55
subepoch: 56
subepoch: 57
subepoch: 58
subepoch: 59
subepoch: 60
subepoch: 61
subepoch: 62
subepoch: 63
subepoch: 64
subepoch: 65
subepoch: 66
subepoch: 67
subepoch: 68
subepoch: 69
subepoch: 70
subepoch: 71
subepoch: 72
subepoch: 73
subepoch: 74
subepoch: 75
subepoch: 76
subepoch:

KeyboardInterrupt: ignored

In [57]:
model.save(epoch=2)

Saving model in ./tf-layers-model/
